## hedging mode does not perform as well as non-hedging 

> If hedging is True, allow trades in both directions simultaneously. If False, the opposite-facing orders first close existing trades in a FIFO manner.

In [ ]:
import pandas as pd
import humanize

In [ ]:
input_path = '../tmp-data/ss_range_bars.final'
known_false_signals = False


In [ ]:
df = pd.read_csv(f'{input_path}.csv', parse_dates=['timestamp'], index_col='timestamp')
df

In [ ]:
def get_start_end_idx_for_period(df: pd.DataFrame, resample_arg: str = 'M') -> str:
        # resample the dataframe into monthly periods
        groups = df.resample(resample_arg)
        # create a list of tuples containing the start and end indices for each period
        start_end_indices = []
        # iterate over each group (period) in the resampled DataFrame
        for period_start, group_df in groups:
            # get the start and end indices of the rows within the current period
            period_start = group_df.index[0]
            period_end = group_df.index[-1]
            # add the start and end indices to the list
            start_end_indices.append((period_start, period_end))
        return start_end_indices

In [ ]:
start_end_indices = get_start_end_idx_for_period(df)
start_end_indices

In [ ]:
from backtesting import Strategy
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

class RangeBarStrategyPreCalc(Strategy):

    per_trade_risk = 0.02
    trades_buy_open_at_time_of_buy = []
    trades_sell_open_at_time_of_sell = []
    trades_open_per_tick = []
    
    
    def init(self):
       self.false_buys = 0
       self.false_sells = 0
       self.trades_buy_open_at_time_of_buy = []
       self.trades_sell_open_at_time_of_sell = []
       self.trades_open_per_tick = []

    # Stop Loss Price = Liquidation Price - (Liquidation Price - Entry Price) / Leverage * Stop Loss Ratio
    def next(self):
        ts = self.data.index[-1]
        current_close = self.data.Close[-1]
        range_size = self.data.average_adr[-1] * 0.1
        potential_profit = self.data.average_adr[-1] * 0.15
        
        pre_calc_signal = self.data.signal[-1]
        pre_calc_false_signal = self.data.false_signal[-1]
    
        known_false_pass = True
        if known_false_signals:
            known_false_pass = pre_calc_false_signal != 1

        order_placed = False
        if pre_calc_signal == 1 and known_false_pass:
            if pre_calc_signal == 1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal buy')
                    self.false_buys += 1
            else:
                print(f'ERROR: buying, against pre_calc_signal')
            order_placed = True        
            self.buy(size=self.per_trade_risk, sl=current_close - range_size, tp=current_close + potential_profit)
        elif pre_calc_signal == -1 and known_false_pass:
            if pre_calc_signal == -1:
                if pre_calc_false_signal == 1: 
                    # print(f'false signal sell')
                    self.false_sells += 1  
            else:
                print(f'ERROR: selling, against pre_calc_signal')
            order_placed = True    
            self.sell(size=self.per_trade_risk, sl=current_close + range_size , tp=current_close - potential_profit)
        if order_placed:    
          is_short_count = len(list(filter(lambda t: t.is_short, self.trades)))
          is_long_count = len(list(filter(lambda t: t.is_long, self.trades)))
          self.trades_buy_open_at_time_of_buy.append(is_long_count)
          self.trades_sell_open_at_time_of_sell.append(is_short_count)
        is_short_count = len(list(filter(lambda t: t.is_short, self.trades)))
        is_long_count = len(list(filter(lambda t: t.is_long, self.trades)))
        self.trades_open_per_tick.append({'timestamp': ts, 'buy': is_long_count, 'sell': is_short_count})

In [ ]:


def backtest_period(strat, df, all_stats, start, end, per_trade_risk=0.02, leverage=20, hedging=False, crypto_cash_adjustment=100000):
    from backtesting import Backtest
    strat.per_trade_risk = per_trade_risk
    bt = Backtest(df, strat, hedging=hedging, cash=crypto_cash_adjustment, margin=1/leverage, commission=(0.04 / 100), exclusive_orders=False)
    stats = bt.run()

    # collect stats
    stats_cp = stats.copy()
    stats_cp['start'] = start
    stats_cp['end'] = end
    all_stats.append(stats_cp)

    # argument stats
    false_buys = stats._strategy.false_buys
    false_sells = stats._strategy.false_sells
    trades = stats['# Trades']
    trade_return_percs = stats._trades['ReturnPct']
    return_percentage = stats['Return [%]']
    win_rate_percentage = stats['Win Rate [%]']
    profit_factor = stats['Profit Factor']
    net_profit = (stats['Equity Final [$]'] - crypto_cash_adjustment)
    equity = stats['Equity Final [$]']
    max_drawdown_percentage = stats['Max. Drawdown [%]']
    average_drawdown_percentage = stats['Avg. Drawdown [%]']
    percentage_false_trades = round(((false_buys + false_sells) / int(trades)) * 100, 2)
    return {
        "trades_buy_open_at_time_of_buy": [i for i in stats._strategy.trades_buy_open_at_time_of_buy if i != 0],
        "trades_sell_open_at_time_of_sell": [i for i in stats._strategy.trades_sell_open_at_time_of_sell if i != 0],
        "trades_open_per_tick": stats._strategy.trades_open_per_tick,
        "trade_return_percentages": trade_return_percs,
        "net_profit": humanize.intcomma(net_profit),
        "start": str(start),
        "end": str(end),
        "return_percentage": return_percentage,
        "win_rate_percentage": win_rate_percentage,
        "profit_factor": profit_factor,
        "max_drawdown_percentage": max_drawdown_percentage,
        "average_drawdown_percentage": average_drawdown_percentage,
        "trades": trades,
        "false_buys": false_buys,
        "false_sells": false_sells,
        "percentage_false_trades": percentage_false_trades,
        "equity": equity,
        "trades": stats._trades,
        "bt": bt
    }

In [ ]:
def tot_prof(trades, initial):
    tot_prof = initial
    for perc in trades:
        tot_prof  *= (1 + perc)
        if tot_prof < 0:
            raise Exception('Negative balance')
    return tot_prof

def test_periods(start_end_indices, all_stats = [], actual_cash=1000, per_trade_risk=0.02, leverage=20, hedging=False, crypto_cash_adjustment=100000):
    profit_takeout = []
    bulk_stats = []
    errors = []
    initial_adjusted_equity = crypto_cash_adjustment
    current_adjusted_equity = initial_adjusted_equity
    for start, end in start_end_indices:
        try:
            df_sample = df[start:end].copy()    
            result = backtest_period(RangeBarStrategyPreCalc, df_sample, all_stats, start, end, per_trade_risk=per_trade_risk, leverage=leverage, hedging=hedging, crypto_cash_adjustment=current_adjusted_equity)
            if result['equity'] < initial_adjusted_equity:
                current_adjusted_equity = result['equity']
            else:
                current_adjusted_equity = initial_adjusted_equity
                actual_equity = tot_prof(result['trade_return_percentages'], actual_cash)
                takeout =  actual_equity - actual_cash
                profit_takeout.append([f'{start}-{end}', takeout])
            bulk_stats.append(result)
        except Exception as e:
            errors.append(f'strat: RangeBarStrategyPreCalc,  start: {str(start)}, end: {str(end)}. error: {str(e)}')

    print(f'errors: {len(errors)}')
    print(f'errors: {errors}')
    df_stats = pd.DataFrame(bulk_stats)
    return df_stats, all_stats, profit_takeout, bulk_stats


In [ ]:
per_trade_risk=0.02
actual_cash=1000
leverage=20

df_stats, all_stats, profit_takeout, bulk_stats = test_periods(start_end_indices, actual_cash=actual_cash,  per_trade_risk=per_trade_risk, leverage=20, hedging=False)
print(f'profit_takeout per period:\n')
for pt in profit_takeout:
    period, profit = pt
    print(f'{period}: {humanize.intcomma(round(profit,2))}')
takeout = [x[1] for x in profit_takeout]
print(f'total profit takeout: {humanize.intcomma(sum(takeout))}')


### hedging=True (0.2, 1000, 20)

2022-04-13 18:27:00-2022-04-30 23:59:00: 406.6  
2022-05-01 00:00:00-2022-05-31 23:54:00: 187,891.65  
2022-06-01 00:00:00-2022-06-30 23:59:01: 262,916.87  
2022-07-01 00:00:00-2022-07-31 23:59:00: 270,078.1  
2022-08-01 00:00:00-2022-08-31 23:46:00: 11,824.93  
2022-09-01 00:00:00-2022-09-30 23:59:00: 15,878.38  
2022-10-01 00:00:00-2022-10-31 23:48:00: 2,992.44  
2022-11-01 00:05:00-2022-11-30 23:54:00: 17,731.77  
2022-12-01 00:05:00-2022-12-31 23:38:01: 2,072.22  
2023-01-01 00:08:00-2023-01-31 23:59:00: 7,556.11  
2023-02-01 00:00:00-2023-02-28 23:34:00: 2,903.24  
2023-03-01 00:16:00-2023-03-31 23:59:00: 10,978.79  
2023-04-01 00:00:00-2023-04-14 16:39:00: 602.97  
total profit takeout: 793,834.0737931037  

### hedging=False (0.2, 1000, 20)

2022-04-13 18:27:00-2022-04-30 23:59:00: 406.6  
2022-05-01 00:00:00-2022-05-31 23:54:00: 184,184.06  
2022-06-01 00:00:00-2022-06-30 23:59:01: 280,256.86  
2022-07-01 00:00:00-2022-07-31 23:59:00: 285,264.94  
2022-08-01 00:00:00-2022-08-31 23:46:00: 11,906.28  
2022-09-01 00:00:00-2022-09-30 23:59:00: 16,999.74  
2022-10-01 00:00:00-2022-10-31 23:48:00: 2,965.89  
2022-11-01 00:05:00-2022-11-30 23:54:00: 16,825.25  
2022-12-01 00:05:00-2022-12-31 23:38:01: 2,073.43  
2023-01-01 00:08:00-2023-01-31 23:59:00: 7,633.88  
2023-02-01 00:00:00-2023-02-28 23:34:00: 3,230.02  
2023-03-01 00:16:00-2023-03-31 23:59:00: 10,978.79  
2023-04-01 00:00:00-2023-04-14 16:39:00: 677.23  
total profit takeout: 823,402.9596548487  



### $100 monthly profits over 1 year @ 2% per trade
2022-04-13 18:27:00-2022-04-30 23:59:00: 40.66  
2022-05-01 00:00:00-2022-05-31 23:54:00: 18,418.41  
2022-06-01 00:00:00-2022-06-30 23:59:01: 28,025.69  
2022-07-01 00:00:00-2022-07-31 23:59:00: 28,526.49  
2022-08-01 00:00:00-2022-08-31 23:46:00: 1,190.63  
2022-09-01 00:00:00-2022-09-30 23:59:00: 1,699.97  
2022-10-01 00:00:00-2022-10-31 23:48:00: 296.59  
2022-11-01 00:05:00-2022-11-30 23:54:00: 1,682.53  
2022-12-01 00:05:00-2022-12-31 23:38:01: 207.34  
2023-01-01 00:08:00-2023-01-31 23:59:00: 763.39  
2023-02-01 00:00:00-2023-02-28 23:34:00: 323.0  
2023-03-01 00:16:00-2023-03-31 23:59:00: 1,097.88  
2023-04-01 00:00:00-2023-04-14 16:39:00: 67.72  
total profit takeout: 82,340.295965485
### $1000 monthly profits over 1 year @ 2% per trade
2022-04-13 18:27:00-2022-04-30 23:59:00: 406.6  
2022-05-01 00:00:00-2022-05-31 23:54:00: 184,184.06  
2022-06-01 00:00:00-2022-06-30 23:59:01: 280,256.86  
2022-07-01 00:00:00-2022-07-31 23:59:00: 285,264.94  
2022-08-01 00:00:00-2022-08-31 23:46:00: 11,906.28  
2022-09-01 00:00:00-2022-09-30 23:59:00: 16,999.74  
2022-10-01 00:00:00-2022-10-31 23:48:00: 2,965.89  
2022-11-01 00:05:00-2022-11-30 23:54:00: 16,825.25  
2022-12-01 00:05:00-2022-12-31 23:38:01: 2,073.43  
2023-01-01 00:08:00-2023-01-31 23:59:00: 7,633.88  
2023-02-01 00:00:00-2023-02-28 23:34:00: 3,230.02  
2023-03-01 00:16:00-2023-03-31 23:59:00: 10,978.79  
2023-04-01 00:00:00-2023-04-14 16:39:00: 677.23  
total profit takeout: 823,402.9596548487
### $1000 monthly profits over 1 year @ 3% per trade
2022-04-13 18:27:00-2022-04-30 23:59:00: 4,594.22  
2022-05-01 00:00:00-2022-05-31 23:54:00: 222,545.35  
2022-06-01 00:00:00-2022-06-30 23:59:01: 280,727.03  
2022-07-01 00:00:00-2022-07-31 23:59:00: 293,289.86  
2022-08-01 00:00:00-2022-08-31 23:46:00: 11,866.61  
2022-09-01 00:00:00-2022-09-30 23:59:00: 17,028.84  
2022-10-01 00:00:00-2022-10-31 23:48:00: 2,988.84  
2022-11-01 00:05:00-2022-11-30 23:54:00: 17,219.16  
2022-12-01 00:05:00-2022-12-31 23:38:01: 2,063.86  
2023-01-01 00:08:00-2023-01-31 23:59:00: 7,620.72  
2023-02-01 00:00:00-2023-02-28 23:34:00: 3,286.28  
2023-03-01 00:16:00-2023-03-31 23:59:00: 10,978.79  
2023-04-01 00:00:00-2023-04-14 16:39:00: 675.22  
total profit takeout: 874,884.7780987181

In [ ]:
for stat in bulk_stats:
    start = stat['start']
    end = stat['end']
    # stat['bt'].plot(filename=f'../tmp-data/period-plots/{start}-{end}')
    stat['trades'].to_csv(f'../tmp-data/period-trades/{actual_cash}-{per_trade_risk}-{start}-{end}.csv')

### sell only volume above limit (535.77 for buy also -- bad idea)

* 873,656.46 volume above 35 @ 7% per trade
* 873,147.50 volume above 0 @ 7% per trade
* 882,868.51 volume above 50 @ 7% per trade
* 946,809.30 volume above 100 @ 7% per trade
* 1,061,383.93 volume above 200 @ 7% per trade
* 6,604,260.28 volume above adv @ 10% per trade
* 22,767.65 volume above adv @ 2% per trade
* 317,835.40 volume above adv @ 5% per trade
* 637,770.41 volume above adv @ 6% per trade

In [ ]:
print(f'all_stats: {len(all_stats)}')
all_stats_str = ""
for stats in all_stats:
    all_stats_str += stats.to_string() + f'\n{stats._trades.to_string()}\n' + "\n\n-------------------------------------------------------------------------------------------------\n\n"

suffix = input_path.split('/')[-1].split('.')[-1]
with open(f'../tmp-data/all_stats-{suffix}-{actual_cash}-{per_trade_risk}-target2.txt', 'w') as f:
    f.write(all_stats_str)    

In [ ]:
pf_measure = df_stats['profit_factor'].sum()
pf_measure

In [ ]:
dd_p_measure = df_stats['max_drawdown_percentage'].sum()
dd_p_measure

## using sell if is_volume_above_adv_limit
### > row['adv'] * 0.75 @ 7% per trade
* dd_p_measure: -280.605
* pf_measure: 32.75
## > row['adv'] @ 10% per trade
* dd_p_measure: -320.35
* pf_measure: 37.74
## > row['adv'] @ 5% per trade
* dd_p_measure: -184.59
* pf_measure: 37.76
## > row['adv'] @ 2% per trade
* dd_p_measure: -70.12
* pf_measure: 36.86

In [ ]:
dict_list_2d = df_stats['trades_open_per_tick']
dict_list_2d

In [ ]:

dict_list_flat = [item for sublist in dict_list_2d for item in sublist]
df_trades_per_tick = pd.DataFrame(dict_list_flat)
df_trades_per_tick['timestamp'] = pd.to_datetime(df_trades_per_tick['timestamp'])
df_trades_per_tick = df_trades_per_tick.set_index('timestamp')
df_trades_per_tick.sort_index(inplace=True)
df_trades_per_tick

In [ ]:
# Remove rows with both 'buy' and 'sell' equal to 0, only if the next row has both columns equal to 0 as well.
remove_indices = []
for i in range(len(df_trades_per_tick) - 1):
    if df_trades_per_tick.iloc[i]['buy'] == 0 and df_trades_per_tick.iloc[i]['sell'] == 0 \
        and df_trades_per_tick.iloc[i+1]['buy'] == 0 and df_trades_per_tick.iloc[i+1]['sell'] == 0:
        remove_indices.append(i)
df_trades_per_tick = df_trades_per_tick.drop(df_trades_per_tick.index[remove_indices])
df_trades_per_tick

In [ ]:
# assuming your DataFrame is named 'df'
mask = (df_trades_per_tick['buy'] > 0) & (df_trades_per_tick['sell'] > 0)
rows_with_both_buy_and_sell = df_trades_per_tick[mask]
rows_with_both_buy_and_sell

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# create a ColumnDataSource from the DataFrame
source = ColumnDataSource(data=dict(
    x=df_trades_per_tick.index,
    buy=df_trades_per_tick['buy'],  # use correct column name
    sell=df_trades_per_tick['sell']  # use correct column name
))

# create a new figure
p = figure(x_axis_type='datetime', width=1400)

# add vertical bars for 'buy' and 'sell'
p.vbar(x='x', top='buy', color='green', alpha=0.5, source=source)  # use correct column name and x-axis column name
p.vbar(x='x', top='sell', color='red', alpha=0.5, source=source)  # use correct column name and x-axis column name

# configure the plot
p.xaxis.axis_label = 'Timestamp'
p.yaxis.axis_label = 'Count'

# display the plot
show(p)


In [ ]:
sorted_df_stats = df_stats.copy()
sorted_df_stats.drop(columns=['trades_open_per_tick'], inplace=True)
# .sort_values(by='profit_factor')
cp =  sorted_df_stats.copy()
trades_buy_open_at_time_of_buy  = cp['trades_buy_open_at_time_of_buy']
trades_sell_open_at_time_of_sell  = cp['trades_sell_open_at_time_of_sell']
sorted_df_stats['trades_sell_open_at_time_of_sell'] = trades_sell_open_at_time_of_sell.apply(lambda trades_sell_open_at_time_of_sell: max(trades_sell_open_at_time_of_sell))
sorted_df_stats['trades_buy_open_at_time_of_buy'] = trades_buy_open_at_time_of_buy.apply(lambda trades_buy_open_at_time_of_buy: max(trades_buy_open_at_time_of_buy))
sorted_df_stats

In [ ]:
import matplotlib.pyplot as plt
import datetime
    
plt.figure()  # create a new figure
for index, row in sorted_df_stats.iterrows():
    x = [datetime.datetime.strptime(index, '%Y-%m-%d %H:%M:%S')] * 2  # convert index to datetime object
    y = [row['trades_open_per_tick'], row['trades_sell_open_per_tick']]
    plt.plot(x, y) 
plt.xlabel('Date and Time')
plt.ylabel('Value')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the array as a line graph
max_open_buy = sorted_df_stats['trades_buy_open_at_time_of_buy'].idxmax()
max_open_sell = sorted_df_stats['trades_sell_open_at_time_of_sell'].idxmax()
plt.plot(trades_buy_open_at_time_of_buy[max_open_buy]) 
plt.plot(trades_sell_open_at_time_of_sell[max_open_sell])
plt.show()
plt.plot(trades_sell_open_at_time_of_sell[max_open_sell] + trades_buy_open_at_time_of_buy[max_open_buy])
plt.show()

In [ ]:
if known_false_signals:
    sorted_df_stats.to_csv(f'{input_path}-{actual_cash}-{per_trade_risk}-monthly-stats_no-false-signals.csv')
else:
    sorted_df_stats.to_csv(f'{input_path}-{actual_cash}-{per_trade_risk}-monthly-stats.csv')